In [1]:
### imports
import json
import import_ipynb
import ipywidgets as widgets
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, fixed, interact_manual
from objects import *
from video import *
from spellcheck import *
from converter import *

### pretty print = off
%pprint

importing Jupyter notebook from objects.ipynb
importing Jupyter notebook from video.ipynb
importing Jupyter notebook from landmarks.ipynb
importing Jupyter notebook from plotting.ipynb
importing Jupyter notebook from spellcheck.ipynb
importing Jupyter notebook from converter.ipynb
Pretty printing has been turned OFF


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Metadata

In [2]:
Meta = metadata('train')
MetaS = metadata('supplemental')

## Selecting datafiles

In [3]:
file = str(Meta.files[2])
Datafile = datafile(Meta, file)

## Selecting videos

In [4]:
vid = Datafile.videos[1]
V = video(Datafile, vid)
print(V)

 Sequence: 198023098
 Phrase: andria duncan
 Signer: 81
 Frames: 42
 Dominant Hand: right
 Percentage: 35.71


In [5]:
f = show_hands(V)
interact(f, frame=widgets.IntSlider(min=0, max=len(V.handframes)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))

interactive(children=(IntSlider(value=0, description='frame', layout=Layout(width='1000px'), max=14), Output()…

<function show_hands.<locals>.show_frame at 0x1098ceef0>

# Label and Pickle 

In [6]:
#Label(V,M)

# change label
def changelabel(self, i, letter):
    self.data_labels[i] = letter
    
# pickle function
def pickle(D,V):
    seq_id = str(V.id)
    Frames = V.frames
    D[seq_id] = V.data_labels
    
#for i in range(81,82):
    #changelabel(V,i,'?')

In [7]:
#for i in range(81,82):
    #changelabel(V,i,'?')
# pickle(D,V)

# Model

### Prepare data

In [8]:
# initialize
X, y = [], []

# loop over file ids
def generate_training_data(file):
    
    # load dictionary
    dstring = 'Labels/Labels' + str(file) + '.json'
    with open( dstring, 'r') as f:
        train = json.load(f)
    
    Met = MetaS if file in MetaS.files else Meta
    
    for file in train:
        # create datafile
        Df = datafile(Met, file)
        videos = train[file]
        for vid in videos:
        
            # load date
            V = video(Df, int(vid))
            N, p = normal_hand_data(V)
            pN = videos[vid]
        
            # loop
            for i in range(len(N)):
                if pN[i] != '?':
                    X.append( N[i] )
                    y.append( Encode( pN[i] ) )


True

In [9]:
# training_files = [105143404, 128822441, 33432165]
# training_files = [ 86446671, 33432165 ]
# for file in training_files:
    # generate_training_data(file)

In [10]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

### Model 

In [11]:
# Model
k = 59
M = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [12]:
# fit model
# M.fit( X, y, epochs = 400, verbose=0 )
# M.fit( X_train, y_train, epochs = 450, verbose=0 )

In [14]:
# Loss, Acctrain = M.evaluate(X, y, verbose=2)
# print('accuracy:', Acctrain)

In [ ]:
#Loss, Acctrain = M.evaluate(X_train, y_train, verbose=2)
#Loss, Acctest = M.evaluate(X_test, y_test, verbose=2)
#print('\nTrain accuracy:', Acctrain)
#print('\nTest accuracy:', Acctest)

 ### save + load past models 

In [ ]:
# save model 
# M.save("alphabetmodel.h5")

# load model 
# M = tf.keras.models.load_model('alphabetmodel.h5')

# External videos

In [28]:
path = 'IMG_0948.MOV'
V = video_file(path)
print(V)

 path: IMG_0948.MOV
 Number of frames: 469
 Percentage of frames with landmarks: 0.6481876332622601


In [29]:
f = show_annotated_video(V)
interact(f, i=widgets.IntSlider(min=0, max=len(V.frames)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))

interactive(children=(IntSlider(value=0, description='i', layout=Layout(width='1000px'), max=468), Output()), …

<function show_annotated_video.<locals>.show_frame at 0x2d42caef0>

### Extra 

In [ ]:
#Good = []
#for i, vid in enumerate(Datafile.videos):
    #V = video(Datafile, vid)
    #p = percentage(V)
    #if p == 100.00: 
        #Good += [vid]
    #print(i,p,vid,len(Good))
    
# Good = np.array(Good)

#Good =[1784587545, 1786414699, 1786455245, 1786973032, 1787094692,
       #1791358393, 1792308520, 1792909008, 1794263510, 1795294511,
       #1797338021, 1797425392, 1799533211, 1801891222, 1802098574,
       #1803766841, 1805308488, 1805565959, 1806516226, 1807475031,
       #1807505088, 1808538895, 1810669537, 1811016240, 1813227209,
       #1814409379, 1814575380, 1815284952, 1815485319]
    
# add to training data
#N, p = normal_hand_data(V)
#pN = V.data_labels

#X, y = [], []

# loop
#for i in range(len(N)):
    #if pN[i] != '?':
        #X += [ N[i] ]
        #y += [ Encode( pN[i] ) ]